# Import Libraries

In [1]:
import sys
sys.path.append("..")
import seaborn as sb
import dgl
import torch

import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from tqdm import tqdm
import torch.nn.functional as F

import statistics
import argparse
import random

from dgl.data import CitationGraphDataset
# from dgl.nn import GINConv, GraphConv, SAGEConv
import seaborn as sb
import torch
import torch.nn as nn
from torch.autograd import Variable
from scipy.optimize import linear_sum_assignment
import scipy
import scipy.optimize
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import sklearn as sk
import networkx as nx


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import random
import math


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data

from torch_geometric.nn import GCNConv,GINConv,SAGEConv,GATConv,PNAConv, GraphSAGE
from torch_geometric.utils import add_self_loops
from torch_geometric.transforms import normalize_features
from pygod.utils import load_data
from pygod.utils.utility import check_parameter
from pygod.metrics import eval_roc_auc
from pygod.generator import gen_contextual_outliers, gen_structural_outliers
from scipy.linalg import sqrtm

/home/koo/miniconda3/envs/gadnr/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/koo/miniconda3/envs/gadnr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/koo/miniconda3/envs/gadnr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  F

# Utils

In [3]:

def _normalize(x):
    x_min = x.min()
    x_max = x.max()
    x_norm = (x - x_min)/x_max
    return x_norm

def gen_joint_structural_outliers(data, m, n, random_state=None):
    """
    We randomly select n nodes from the network which will be the anomalies 
    and for each node we select m nodes from the network. 
    We connect each of n nodes with the m other nodes.

    Parameters
    ----------
    data : PyTorch Geometric Data instance (torch_geometric.data.Data)
        The input data.
    m : int
        Number nodes in the outlier cliques.
    n : int
        Number of outlier cliques.
    p : int, optional
        Probability of edge drop in cliques. Default: ``0``.
    random_state : int, optional
        The seed to control the randomness, Default: ``None``.

    Returns
    -------
    data : PyTorch Geometric Data instance (torch_geometric.data.Data)
        The structural outlier graph with injected edges.
    y_outlier : torch.Tensor
        The outlier label tensor where 1 represents outliers and 0 represents
        regular nodes.
    """

    if not isinstance(data, Data):
        raise TypeError("data should be torch_geometric.data.Data")

    if isinstance(m, int):
        check_parameter(m, low=0, high=data.num_nodes, param_name='m')
    else:
        raise ValueError("m should be int, got %s" % m)

    if isinstance(n, int):
        check_parameter(n, low=0, high=data.num_nodes, param_name='n')
    else:
        raise ValueError("n should be int, got %s" % n)

    check_parameter(m * n, low=0, high=data.num_nodes, param_name='m*n')

    if random_state:
        np.random.seed(random_state)


    outlier_idx = np.random.choice(data.num_nodes, size=n, replace=False)
    all_nodes = [i for i in range(data.num_nodes)]
    rem_nodes = []
    
    for node in all_nodes:
        if node is not outlier_idx:
            rem_nodes.append(node)
    
    
    
    new_edges = []
    
    # connect all m nodes in each clique
    for i in range(0, n):
        other_idx = np.random.choice(data.num_nodes, size=m, replace=False)
        for j in other_idx:
            new_edges.append(torch.tensor([[i, j]], dtype=torch.long))
                    

    new_edges = torch.cat(new_edges)


    y_outlier = torch.zeros(data.x.shape[0], dtype=torch.long)
    y_outlier[torch.tensor(outlier_idx, dtype=torch.long)] = 1


    data.edge_index = torch.cat([data.edge_index, new_edges.T], dim=1)

    return data, y_outlier


def KL_neighbor_loss(predictions, targets, mask_len):
    x1 = predictions.squeeze().cpu().detach()
    x2 = targets.squeeze().cpu().detach()
    
    mean_x1 = x1.mean(0)
    mean_x2 = x2.mean(0)
    
    nn = x1.shape[0]
    h_dim = x1.shape[1]
    
    cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / max((nn-1),1)
    cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / max((nn-1),1)
    
    eye = torch.eye(h_dim)
    cov_x1 = cov_x1 + eye
    cov_x2 = cov_x2 + eye
    
    KL_loss = 0.5 * (math.log(torch.det(cov_x1) / torch.det(cov_x2)) - h_dim  + torch.trace(torch.inverse(cov_x2).matmul(cov_x1)) 
            + (mean_x2 - mean_x1).reshape(1,-1).matmul(torch.inverse(cov_x2)).matmul(mean_x2 - mean_x1))
    KL_loss = KL_loss.to(device)
    return KL_loss

def W2_neighbor_loss(predictions, targets, mask_len):
    
    x1 = predictions.squeeze().cpu().detach()
    x2 = targets.squeeze().cpu().detach()
    
    mean_x1 = x1.mean(0)
    mean_x2 = x2.mean(0)

    nn = x1.shape[0]
    
    cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / (nn-1)
    cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / (nn-1)
    

    W2_loss = torch.square(mean_x1-mean_x2).sum() + torch.trace(cov_x1 + cov_x2 
                     + 2 * sqrtm(sqrtm(cov_x1) @ (cov_x2.numpy()) @ (sqrtm(cov_x1))))

    return W2_loss



# Layers

In [4]:
class MLP(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, output_dim):
        

        super(MLP, self).__init__()

        self.linear_or_not = True  # default is linear model
        self.num_layers = num_layers

        if num_layers < 1:
            raise ValueError("number of layers should be positive!")
        elif num_layers == 1:
            # Linear model
            self.linear = nn.Linear(input_dim, output_dim)
        else:
            # Multi-layer model
            self.linear_or_not = False
            self.linears = torch.nn.ModuleList()
            self.batch_norms = torch.nn.ModuleList()

            self.linears.append(nn.Linear(input_dim, hidden_dim))
            for layer in range(num_layers - 2):
                self.linears.append(nn.Linear(hidden_dim, hidden_dim))
            self.linears.append(nn.Linear(hidden_dim, output_dim))

            for layer in range(num_layers - 1):
                self.batch_norms.append(nn.BatchNorm1d((hidden_dim)))

    def forward(self, x):
        if self.linear_or_not:
            # If linear model
            return self.linear(x)
        else:
            # If MLP
            h = x
            for layer in range(self.num_layers - 1):
                h = self.linears[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 0, 1)
                    h = torch.transpose(h, 1, 2)
                    
                h = self.batch_norms[layer](h)
                
                if len(h.shape) > 2:
                    h = torch.transpose(h, 1, 2)
                    h = torch.transpose(h, 0, 1)

                h = F.relu(h)
                # h = F.relu(self.linears[layer](h))
                
            return self.linears[self.num_layers - 1](h)


class MLP_generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP_generator, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.linear2 = nn.Linear(output_dim, output_dim)
        self.linear3 = nn.Linear(output_dim, output_dim)
        self.linear4 = nn.Linear(output_dim, output_dim)

    def forward(self, embedding):
        neighbor_embedding = F.relu(self.linear(embedding))
        neighbor_embedding = F.relu(self.linear2(neighbor_embedding))
        neighbor_embedding = F.relu(self.linear3(neighbor_embedding))
        neighbor_embedding = self.linear4(neighbor_embedding)
        return neighbor_embedding


class PairNorm(nn.Module):
    def __init__(self, mode='PN', scale=10):
        
        assert mode in ['None', 'PN', 'PN-SI', 'PN-SCS']
        super(PairNorm, self).__init__()
        self.mode = mode
        self.scale = scale

        # Scale can be set based on origina data, and also the current feature lengths.
        # We leave the experiments to future. A good pool we used for choosing scale:
        # [0.1, 1, 10, 50, 100]
    def forward(self, x):
        if self.mode == 'None':
            return x
        col_mean = x.mean(dim=0)
        if self.mode == 'PN':
            x = x - col_mean
            rownorm_mean = (1e-6 + x.pow(2).sum(dim=1).mean()).sqrt()
            x = self.scale * x / rownorm_mean
        if self.mode == 'PN-SI':
            x = x - col_mean
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual
        if self.mode == 'PN-SCS':
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual - col_mean
        return x


# FNN
class FNN(nn.Module):
    def __init__(self, in_features, hidden, out_features, layer_num):
        super(FNN, self).__init__()
        self.linear1 = MLP(layer_num, in_features, hidden, out_features)
        self.linear2 = nn.Linear(out_features, out_features)
    def forward(self, embedding):
        x = self.linear1(embedding)
        x = self.linear2(F.relu(x))
        x = F.relu(x)
        return x


# GAD-NR

In [16]:
def safe_normalize(tensor):
    max_val = torch.max(tensor)
    min_val = torch.min(tensor)
    denom = max_val - min_val
    if denom == 0:
        return torch.zeros_like(tensor)  # or tensor.clone() * 0
    else:
        return (tensor - min_val) / denom

In [20]:

# Training
def train(data, y, yc, ys, yj, ysj, lr, epoch, device, encoder, lambda_loss1, lambda_loss2, lambda_loss3, hidden_dim, sample_size=10,loss_step=20,
          real_loss=False,calculate_contextual=False,calculate_structural=False):
    '''
     Main training function
     INPUT:
     -----------------------
     data : torch geometric dataset object
     lr    :    learning rate
     epoch     :    number of training epoch
     device     :   CPU or GPU
     encoder    :    GCN or GIN or GraphSAGE
     lambda_loss    :   Trade-off between degree loss and neighborhood reconstruction loss
     hidden_dim     :   latent variable dimension
    '''
    
    in_nodes = data.edge_index[0,:]
    out_nodes = data.edge_index[1,:]
    
    
    neighbor_dict = {}
    for in_node, out_node in zip(in_nodes, out_nodes):
        if in_node.item() not in neighbor_dict:
            neighbor_dict[in_node.item()] = []
        neighbor_dict[in_node.item()].append(out_node.item())

    neighbor_num_list = []
    for i in neighbor_dict:
        neighbor_num_list.append(len(neighbor_dict[i]))
    
    neighbor_num_list = torch.tensor(neighbor_num_list).to(device)
    
    in_dim = data.x.shape[1]
    GNNModel = GNNStructEncoder(in_dim, hidden_dim, hidden_dim, 2, sample_size, device=device, 
                    neighbor_num_list=neighbor_num_list, GNN_name=encoder, 
                    lambda_loss1=lambda_loss1, lambda_loss2=lambda_loss2,lambda_loss3=lambda_loss3)
    GNNModel.to(device)
    degree_params = list(map(id, GNNModel.degree_decoder.parameters()))
    base_params = filter(lambda p: id(p) not in degree_params,
                         GNNModel.parameters())

    opt = torch.optim.Adam([{'params': base_params}, {'params': GNNModel.degree_decoder.parameters(), 'lr': 1e-2}],lr=lr, weight_decay=0.0003)
    min_loss = float('inf')
    arg_min_loss_per_node = None
    
    best_auc = 0
    best_auc_contextual = 0
    best_auc_dense_structural = 0
    best_auc_joint_structural = 0
    best_auc_structure_type = 0
    
        
    loss_values = []
    for i in tqdm(range(1,epoch+1,1)):
        
        
        if i%loss_step==0:
            GNNModel.lambda_loss2 = GNNModel.lambda_loss2 + 0.5
            GNNModel.lambda_loss3 = GNNModel.lambda_loss3 / 2
        
        loss,loss_per_node,h_loss,degree_loss,feature_loss = GNNModel(data.edge_index, data.x, neighbor_num_list, neighbor_dict, device=device)
        
        
        
        loss_per_node = loss_per_node.cpu().detach()
        
        # h_loss = h_loss.cpu().detach()
        # degree_loss = degree_loss.cpu().detach()
        # feature_loss = feature_loss.cpu().detach()


        h_loss_norm = safe_normalize(h_loss.cpu().detach())
        degree_loss_norm = safe_normalize(degree_loss.cpu().detach())
        feature_loss_norm = safe_normalize(feature_loss.cpu().detach())

        
        h_loss_norm = h_loss / (torch.max(h_loss) - torch.min(h_loss))
        degree_loss_norm = degree_loss / (torch.max(degree_loss) - torch.min(degree_loss))
        feature_loss_norm = feature_loss / (torch.max(feature_loss) - torch.min(feature_loss))
        
        comb_loss = args.h_loss_weight * h_loss_norm + args.degree_loss_weight *  degree_loss_norm + args.feature_loss_weight * feature_loss_norm
        
        if real_loss:
            comp_loss = loss_per_node
        else:
            comp_loss = comb_loss
            
        
        auc_score = eval_roc_auc(y.cpu().detach().tolist(), comp_loss.cpu().detach().tolist())
        print("Dataset Name: ",dataset_str, ", AUC Score(benchmark/combined): ", auc_score)
        
        contextual_auc_score = eval_roc_auc(yc.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (contextual): ", contextual_auc_score)

        dense_structural_auc_score = eval_roc_auc(ys.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (structural): ", dense_structural_auc_score)
        
        joint_type_auc_score = eval_roc_auc(yj.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (joint-type): ", joint_type_auc_score)
        
        structure_type_auc_score = eval_roc_auc(ysj.cpu().detach().tolist(), comp_loss.cpu().detach().tolist()) * 100
        print("Dataset Name: ",dataset_str, ", AUC Score (structure type): ", structure_type_auc_score) 
        
        best_auc = max(best_auc, auc_score)
        best_auc_contextual = max(best_auc_contextual, contextual_auc_score)
        best_auc_dense_structural = max(best_auc_dense_structural, dense_structural_auc_score)
        best_auc_joint_type = max(best_auc_joint_structural, joint_type_auc_score)
        best_auc_structure_type = max(best_auc_structure_type, structure_type_auc_score)
        
        
        
        print("===========================================================================================")
        print("Dataset Name: ",dataset_str, " Best AUC Score(benchmark/combined): ", best_auc)
        
        print("Dataset Name: ",dataset_str, " Best AUC Score (contextual): ", best_auc_contextual)

        
        print("Dataset Name: ",dataset_str, " Best AUC Score (structural): ", best_auc_dense_structural)
        
        
        print("Dataset Name: ",dataset_str, " Best AUC Score (joint-type): ", best_auc_joint_type)
        
        
        print("Dataset Name: ",dataset_str, " Best AUC Score (structure type): ", best_auc_structure_type) 
        print("===========================================================================================")
        
        
        if loss < min_loss:
            min_loss = loss
            arg_min_loss_per_node = loss_per_node
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        
        
        loss = loss.cpu().detach()
        # loss_values.append(loss)
        loss_values.append(loss.item())  # 또는 .tolist()

        
        
        if args.plot_loss:

            plt.plot(np.array(loss_values), 'r')
            plt.show()
    
    return min_loss.item(), arg_min_loss_per_node.cpu().detach()




def evaluate(model, embeddings, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(embeddings)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train_real_datasets(dataset_str, epoch_num = 10, lr = 5e-6, encoder = "GCN", 
                        lambda_loss1=1e-2, lambda_loss2=1e-3, lambda_loss3=1e-3, sample_size=8, loss_step=20, hidden_dim=None,
                        real_loss=False,calculate_contextual=False,calculate_structural=False):
    
    data = load_data(dataset_str)
    node_features = data.x
    
    if args.normalize_feat:
    
        node_features_min = node_features.min()
        node_features_max = node_features.max()
        node_features = (node_features - node_features_min)/node_features_max
        data.x = node_features
    
    yc = []
    ys = []
    yj = []
    
    if calculate_contextual:
        
        if dataset_str == "inj_cora":
            yc = data.y >> 0 & 1 # contextual outliers
        else:
            data, yc = gen_contextual_outliers(data=data,n=args.contextual_n,k=args.contextual_k)
            
        yc = yc.cpu().detach()
    
    
    if calculate_structural:
        
        if dataset_str == "inj_cora":
            ys = data.y >> 1 & 1 # structural outliers
        else:
            data, ys = gen_structural_outliers(data=data,n=args.structural_n,m=args.structural_m,p=0.2)
            
        ys = ys.cpu().detach()
        data, yj = gen_joint_structural_outliers(data=data,n=args.structural_n,m=args.structural_m)
        
    
    if args.use_combine_outlier:
        data.y = torch.logical_or(ys, yc).int()
        
    ysj = torch.logical_or(ys, yj).int()
    y = data.y.bool()    # binary labels (inlier/outlier)
    y = y.cpu().detach()
    
    edge_index = data.edge_index.cpu()
    
    num_nodes = node_features.shape[0]
    self_edges = torch.tensor([[i for i in range(num_nodes)],[i for i in range(num_nodes)]])
    edge_index = torch.cat([edge_index,self_edges],dim=1)
    data.edge_index = edge_index
    data = data.to(device)
    

    loss, loss_per_node, = train(data, y, yc, ys, yj, ysj, lr=lr, epoch=epoch_num, device=device, encoder=encoder, lambda_loss1=lambda_loss1, 
          lambda_loss2=lambda_loss2, lambda_loss3=lambda_loss3, hidden_dim=hidden_dim, sample_size=sample_size,loss_step=loss_step,
                                 real_loss=real_loss,calculate_contextual=calculate_contextual,calculate_structural=calculate_structural)

In [21]:
# generate ground truth neighbors Hv
def generate_gt_neighbor(neighbor_dict, node_embeddings, neighbor_num_list, in_dim):
    max_neighbor_num = max(neighbor_num_list)
    all_gt_neighbor_embeddings = []
    for i, embedding in enumerate(node_embeddings):
        neighbor_indexes = neighbor_dict[i]
        neighbor_embeddings = []
        for index in neighbor_indexes:
            neighbor_embeddings.append(node_embeddings[index].tolist())
        if len(neighbor_embeddings) < max_neighbor_num:
            for _ in range(max_neighbor_num - len(neighbor_embeddings)):
                neighbor_embeddings.append(torch.zeros(in_dim).tolist())
        all_gt_neighbor_embeddings.append(neighbor_embeddings)
    return all_gt_neighbor_embeddings


# Main Autoencoder structure here
class GNNStructEncoder(nn.Module):
    def __init__(self, in_dim0, in_dim, hidden_dim, layer_num, sample_size, device, neighbor_num_list, 
                 GNN_name="GIN", norm_mode="PN-SCS", norm_scale=20, lambda_loss1=0.01, lambda_loss2=0.001, lambda_loss3=0.0001):
        
        super(GNNStructEncoder, self).__init__()
        
        self.mlp0 = nn.Linear(in_dim0, hidden_dim)
        self.norm = PairNorm(norm_mode, norm_scale)
        self.out_dim = hidden_dim
        self.lambda_loss1 = lambda_loss1
        self.lambda_loss2 = lambda_loss2
        self.lambda_loss3 = lambda_loss3
        # GNN Encoder
        if GNN_name == "GIN":
            self.linear1 = MLP(layer_num, hidden_dim, hidden_dim, hidden_dim)
            self.graphconv1 = GINConv(self.linear1)
            self.linear2 = MLP(layer_num, hidden_dim, hidden_dim, hidden_dim)
            self.graphconv2 = GINConv(self.linear2)
        elif GNN_name == "GCN":
            self.graphconv1 = GCNConv(hidden_dim, hidden_dim)
            self.graphconv2 = GCNConv(hidden_dim, hidden_dim)
        elif GNN_name == "GAT":
            self.graphconv1 = GATConv(hidden_dim, hidden_dim)
            self.graphconv2 = GATConv(hidden_dim, hidden_dim)
        else:
            self.graphconv1 = SAGEConv(hidden_dim, hidden_dim, aggr='sum')
            # self.graphconv2 = SAGEConv(hidden_dim, hidden_dim, aggr='mean')
            
            # self.graphconv1 = GraphSAGE(hidden_dim, hidden_dim, aggr='mean', num_layers=1)
            

        self.neighbor_num_list = neighbor_num_list
        self.tot_node = len(neighbor_num_list)

        self.gaussian_mean = nn.Parameter(
            torch.FloatTensor(sample_size, hidden_dim).uniform_(-0.5 / hidden_dim,
                                                                                     0.5 / hidden_dim)).to(device)
        self.gaussian_log_sigma = nn.Parameter(
            torch.FloatTensor(sample_size, hidden_dim).uniform_(-0.5 / hidden_dim,
                                                                                     0.5 / hidden_dim)).to(device)
        self.m = torch.distributions.Normal(torch.zeros(sample_size, hidden_dim),
                                            torch.ones(sample_size, hidden_dim))
        
        self.m_batched = torch.distributions.Normal(torch.zeros(sample_size, self.tot_node, hidden_dim),
                                            torch.ones(sample_size, self.tot_node, hidden_dim))

        self.m_h = torch.distributions.Normal(torch.zeros(sample_size, hidden_dim),
                                            50* torch.ones(sample_size, hidden_dim))

        # Before MLP Gaussian Means, and std

        self.mlp_gaussian_mean = nn.Parameter(
            torch.FloatTensor(hidden_dim).uniform_(-0.5 / hidden_dim, 0.5 / hidden_dim)).to(device)
        self.mlp_gaussian_log_sigma = nn.Parameter(
            torch.FloatTensor(hidden_dim).uniform_(-0.5 / hidden_dim, 0.5 / hidden_dim)).to(device)
        self.mlp_m = torch.distributions.Normal(torch.zeros(hidden_dim), torch.ones(hidden_dim))

        self.mlp_mean = FNN(hidden_dim, hidden_dim, hidden_dim, 3)
        self.mlp_sigma = FNN(hidden_dim, hidden_dim, hidden_dim, 3)
        self.softplus = nn.Softplus()

        self.mean_agg = SAGEConv(hidden_dim, hidden_dim, aggr='mean', normalize = False)
        # self.mean_agg = GraphSAGE(hidden_dim, hidden_dim, aggr='mean', num_layers=1)
        self.std_agg = PNAConv(hidden_dim, hidden_dim, aggregators=["std"],scalers=["identity"], deg=neighbor_num_list)        
        self.layer1_generator = MLP_generator(hidden_dim, hidden_dim)
        
        # Decoders
        self.degree_decoder = FNN(hidden_dim, hidden_dim, 1, 4)
        self.feature_decoder = FNN(hidden_dim, hidden_dim, in_dim, 3)
        self.degree_loss_func = nn.MSELoss()
        self.feature_loss_func = nn.MSELoss()
        self.pool = mp.Pool(4)
        self.in_dim = in_dim
        self.sample_size = sample_size 
        self.init_projection = FNN(in_dim, hidden_dim, hidden_dim, 1)
        

    def forward_encoder(self, x, edge_index):
        
        # Apply graph convolution and activation, pair-norm to avoid trivial solution
        h0 = self.mlp0(x)
        l1 = self.graphconv1(h0, edge_index)
        return l1, h0
        
        

    # Sample neighbors from neighbor set, if the length of neighbor set less than sample size, then do the padding.
    def sample_neighbors(self, indexes, neighbor_dict, gt_embeddings):
        sampled_embeddings_list = []
        mark_len_list = []
        for index in indexes:
            sampled_embeddings = []
            neighbor_indexes = neighbor_dict[index]
            if len(neighbor_indexes) < self.sample_size:
                mask_len = len(neighbor_indexes)
                sample_indexes = neighbor_indexes
            else:
                sample_indexes = random.sample(neighbor_indexes, self.sample_size)
                mask_len = self.sample_size
            for index in sample_indexes:
                sampled_embeddings.append(gt_embeddings[index].tolist())
            if len(sampled_embeddings) < self.sample_size:
                for _ in range(self.sample_size - len(sampled_embeddings)):
                    sampled_embeddings.append(torch.zeros(self.out_dim).tolist())
            sampled_embeddings_list.append(sampled_embeddings)
            mark_len_list.append(mask_len)
        
        return sampled_embeddings_list, mark_len_list
    
    def reconstruction_neighbors2(self, l1, h0, edge_index):
                
        recon_loss = 0
        recon_loss_per_node = []
    
        sample_sz_per_node = [self.sample_size]* self.tot_node

        # mean_neigh = self.graphconv1(h0, edge_index)
        mean_neigh = self.mean_agg(h0, edge_index).detach()
        std_neigh = self.std_agg(h0, edge_index).detach()
        # mean_neigh = self.graphconv2(mean_neigh, edge_index)
        # mean_neigh = l1
        # mean_neigh = self.mean_agg(h0, edge_index, num_sampled_nodes_per_hop=sample_sz_per_node)
        
        
        cov_neigh = torch.bmm(std_neigh.unsqueeze(dim=-1),std_neigh.unsqueeze(dim=1))
        
        target_mean = mean_neigh
        target_cov = cov_neigh
        
        self_embedding = l1
        # self_embedding = _normalize(self_embedding)
        
        self_embedding = self_embedding.unsqueeze(0)
        self_embedding = self_embedding.repeat(self.sample_size, 1, 1)
        generated_mean = self.mlp_mean(self_embedding)
        generated_sigma = self.mlp_sigma(self_embedding)

        
        std_z = self.m_batched.sample().to(device)
        var = generated_mean + generated_sigma.exp() * std_z
        nhij = self.layer1_generator(var)
        
        generated_mean = torch.mean(nhij,dim=0)
        generated_std = torch.std(nhij,dim=0)
        generated_cov = torch.bmm(generated_std.unsqueeze(dim=-1),generated_std.unsqueeze(dim=1))/self.sample_size
        
        
        tot_nodes = l1.shape[0]
        h_dim = l1.shape[1]
        
        single_eye = torch.eye(h_dim).to(device)
        single_eye = single_eye.unsqueeze(dim=0)
        batch_eye = single_eye.repeat(tot_nodes,1,1)
        
        target_cov = target_cov + batch_eye
        generated_cov = generated_cov + batch_eye

        
        det_target_cov = torch.linalg.det(target_cov) 
        det_generated_cov = torch.linalg.det(generated_cov) 
        trace_mat = torch.matmul(torch.inverse(generated_cov),target_cov)
             
        
        x = torch.bmm(torch.unsqueeze(generated_mean - target_mean,dim=1),torch.inverse(generated_cov))
        y = torch.unsqueeze(generated_mean - target_mean,dim=-1)
        z = torch.bmm(x,y).squeeze()
        
        KL_loss = 0.5 * (torch.log(det_target_cov / det_generated_cov) - h_dim  + trace_mat.diagonal(offset=0, dim1=-1, dim2=-2).sum(-1) + z)
        
        recon_loss = torch.mean(KL_loss)
        recon_loss_per_node = KL_loss
        
        
        return recon_loss, recon_loss_per_node
    
                
        # self_embedding_min = self_embedding.min()
        # self_embedding_max = self_embedding.max()
        # self_embedding = (self_embedding - self_embedding_min)/(self_embedding_max+1e-14)
        # generated_mean = self.mean_generator(self_embedding)
        # generated_mean = _normalize(generated_mean)
        # generated_std = self.std_generator(self_embedding)
        # generated_std = _normalize(generated_std)
        # generated_std = generated_std.abs().squeeze()
        # generated_std = self.softplus(generated_std)
        # generated_std = generated_std.exp().squeeze()
        
        # cov_x1 = (x1-mean_x1).transpose(1,0).matmul(x1-mean_x1) / max((nn-1),1)
        # cov_x2 = (x2-mean_x2).transpose(1,0).matmul(x2-mean_x2) / max((nn-1),1)
        
        # generated_cov = generated_cov + torch.rand(generated_cov.shape).to(device)
        # generated_cov = generated_cov * batch_eye
        
        # print(torch.isfinite(det_target_cov).all())
        # print(torch.isfinite(det_generated_cov).all()) 
        
        # KL_loss = 0.5 * (math.log(torch.det(cov_x1) / torch.det(cov_x2)) - h_dim  + torch.trace(torch.inverse(cov_x2).matmul(cov_x1)) 
        #     + (mean_x2 - mean_x1).reshape(1,-1).matmul(torch.inverse(cov_x2)).matmul(mean_x2 - mean_x1))
    

    def reconstruction_neighbors(self, FNN_generator, neighbor_indexes, neighbor_dict, from_layer, to_layer, device):
        
        
        local_index_loss = 0
        local_index_loss_per_node = []
        sampled_embeddings_list, mark_len_list = self.sample_neighbors(neighbor_indexes, neighbor_dict, to_layer)
        for i, neighbor_embeddings1 in enumerate(sampled_embeddings_list):
            # Generating h^k_v, reparameterization trick
            
            # print(len(neighbor_embeddings1))
            
            index = neighbor_indexes[i]
            mask_len1 = mark_len_list[i]
            mean = from_layer[index].repeat(self.sample_size, 1)
            mean = self.mlp_mean(mean)
            sigma = from_layer[index].repeat(self.sample_size, 1)
            sigma = self.mlp_sigma(sigma)
            std_z = self.m.sample().to(device)
            var = mean + sigma.exp() * std_z
            nhij = FNN_generator(var)
            
            generated_neighbors = nhij
            sum_neighbor_norm = 0
            
            for indexi, generated_neighbor in enumerate(generated_neighbors):
                sum_neighbor_norm += torch.norm(generated_neighbor) / math.sqrt(self.out_dim)
            generated_neighbors = torch.unsqueeze(generated_neighbors, dim=0).to(device)
            target_neighbors = torch.unsqueeze(torch.FloatTensor(neighbor_embeddings1), dim=0).to(device)
            
            if args.neigh_loss == "KL":
                KL_loss = KL_neighbor_loss(generated_neighbors, target_neighbors, mask_len1)
                local_index_loss += KL_loss
                local_index_loss_per_node.append(KL_loss)
            
            else:
                W2_loss = W2_neighbor_loss(generated_neighbors, target_neighbors, mask_len1)
                local_index_loss += W2_loss
                local_index_loss_per_node.append(W2_loss)
            
            
        local_index_loss_per_node = torch.stack(local_index_loss_per_node)
        return local_index_loss, local_index_loss_per_node
    

    def neighbor_decoder(self, gij, ground_truth_degree_matrix, h0, neighbor_dict, device, h, edge_index):
        
        # Degree decoder below:
        tot_nodes = gij.shape[0]
        degree_logits = self.degree_decoding(gij)
        ground_truth_degree_matrix = torch.unsqueeze(ground_truth_degree_matrix, dim=1)
        degree_loss = self.degree_loss_func(degree_logits, ground_truth_degree_matrix.float())
        degree_loss_per_node = (degree_logits-ground_truth_degree_matrix).pow(2)
        _, degree_masks = torch.max(degree_logits.data, dim=1)
        h_loss = 0
        feature_loss = 0
        # layer 1
        loss_list = []
        loss_list_per_node = []
        feature_loss_list = []
        # Sample multiple times to remove noise
        for _ in range(3):
            local_index_loss_sum = 0
            local_index_loss_sum_per_node = []
            indexes = []
            h0_prime = self.feature_decoder(gij)
            feature_losses = self.feature_loss_func(h0, h0_prime)
            feature_losses_per_node = (h0-h0_prime).pow(2).mean(1)
            feature_loss_list.append(feature_losses_per_node)
            

            
            # local_index_loss, local_index_loss_per_node = self.reconstruction_neighbors(self.layer1_generator, indexes, neighbor_dict, gij, h0, device)
            local_index_loss, local_index_loss_per_node = self.reconstruction_neighbors2(gij,h0,edge_index)
            
            # print(local_index_loss, local_index_loss2)
            # print(local_index_loss_per_node, local_index_loss_per_node2)

            loss_list.append(local_index_loss)
            loss_list_per_node.append(local_index_loss_per_node)
            
        loss_list = torch.stack(loss_list)
        h_loss += torch.mean(loss_list)
        
        loss_list_per_node = torch.stack(loss_list_per_node)
        h_loss_per_node = torch.mean(loss_list_per_node,dim=0)
        
        feature_loss_per_node = torch.mean(torch.stack(feature_loss_list),dim=0)
        feature_loss += torch.mean(torch.stack(feature_loss_list))
                
        h_loss_per_node = h_loss_per_node.reshape(tot_nodes,1)
        degree_loss_per_node = degree_loss_per_node.reshape(tot_nodes,1)
        feature_loss_per_node = feature_loss_per_node.reshape(tot_nodes,1)
        
        loss = self.lambda_loss1 * h_loss + degree_loss * self.lambda_loss3 + self.lambda_loss2 * feature_loss
        loss_per_node = self.lambda_loss1 * h_loss_per_node + degree_loss_per_node * self.lambda_loss3 + self.lambda_loss2 * feature_loss_per_node
        

        return loss,loss_per_node,h_loss_per_node,degree_loss_per_node,feature_loss_per_node

    def degree_decoding(self, node_embeddings):
        degree_logits = F.relu(self.degree_decoder(node_embeddings))
        return degree_logits

    def forward(self, edge_index, x, ground_truth_degree_matrix, neighbor_dict, device):
        
        # Generate GNN encodings
        l1, h0 = self.forward_encoder(x, edge_index)
        loss, loss_per_node,h_loss,degree_loss,feature_loss = self.neighbor_decoder(l1, ground_truth_degree_matrix, h0, neighbor_dict, device, x, edge_index)
        
        return loss, loss_per_node,h_loss,degree_loss,feature_loss


# Execution

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="disney")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=5000)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.5) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])


print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  disney lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 5000 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.5 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  1%|          | 5/500 [00:01<01:28,  5.60it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.4336158192090395
Dataset Name:  disney , AUC Score (contextual):  39.94490358126722
Dataset Name:  disney , AUC Score (structural):  46.727828746177366
Dataset Name:  disney , AUC Score (joint-type):  30.02754820936639
Dataset Name:  disney , AUC Score (structure type):  43.39622641509434
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.4336158192090395
Dataset Name:  disney  Best AUC Score (contextual):  39.94490358126722
Dataset Name:  disney  Best AUC Score (structural):  46.727828746177366
Dataset Name:  disney  Best AUC Score (joint-type):  30.02754820936639
Dataset Name:  disney  Best AUC Score (structure type):  43.39622641509434
Dataset Name:  disney , AUC Score(benchmark/combined):  0.4350282485875706
Dataset Name:  disney , AUC Score (contextual):  39.94490358126722
Dataset Name:  disney , AUC Score (structural):  46.54434250764526
Dataset Name:  disney , AUC Score (joint-type):  30.303030303030305
Dataset

  2%|▏         | 11/500 [00:01<00:38, 12.57it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.42655367231638414
Dataset Name:  disney , AUC Score (contextual):  39.11845730027548
Dataset Name:  disney , AUC Score (structural):  48.50152905198777
Dataset Name:  disney , AUC Score (joint-type):  27.548209366391184
Dataset Name:  disney , AUC Score (structure type):  44.44444444444445
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.43644067796610164
Dataset Name:  disney  Best AUC Score (contextual):  40.22038567493113
Dataset Name:  disney  Best AUC Score (structural):  48.50152905198777
Dataset Name:  disney  Best AUC Score (joint-type):  27.548209366391184
Dataset Name:  disney  Best AUC Score (structure type):  44.44444444444445
Dataset Name:  disney , AUC Score(benchmark/combined):  0.43220338983050843
Dataset Name:  disney , AUC Score (contextual):  39.94490358126721
Dataset Name:  disney , AUC Score (structural):  49.235474006116206
Dataset Name:  disney , AUC Score (joint-type):  27.27272727272727
Data

...

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6186440677966101
Dataset Name:  disney , AUC Score (contextual):  23.966942148760324
Dataset Name:  disney , AUC Score (structural):  32.8440366972477
Dataset Name:  disney , AUC Score (joint-type):  46.005509641873275
Dataset Name:  disney , AUC Score (structure type):  34.538784067085956
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7754237288135593
Dataset Name:  disney  Best AUC Score (contextual):  51.790633608815426
Dataset Name:  disney  Best AUC Score (structural):  60.24464831804281
Dataset Name:  disney  Best AUC Score (joint-type):  46.005509641873275
Dataset Name:  disney  Best AUC Score (structure type):  59.11949685534591
Dataset Name:  disney , AUC Score(benchmark/combined):  0.6440677966101694
Dataset Name:  disney , AUC Score (contextual):  30.02754820936639
Dataset Name:  disney , AUC Score (structural):  39.388379204892956
Dataset Name:  disney , AUC Score (joint-type):  48.760330578512395
Data

 99%|█████████▉| 494/500 [00:16<00:00, 35.45it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6101694915254238
Dataset Name:  disney , AUC Score (contextual):  28.37465564738292
Dataset Name:  disney , AUC Score (structural):  37.12538226299694
Dataset Name:  disney , AUC Score (joint-type):  32.231404958677686
Dataset Name:  disney , AUC Score (structure type):  35.58700209643605
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7754237288135593
Dataset Name:  disney  Best AUC Score (contextual):  51.790633608815426
Dataset Name:  disney  Best AUC Score (structural):  60.24464831804281
Dataset Name:  disney  Best AUC Score (joint-type):  32.231404958677686
Dataset Name:  disney  Best AUC Score (structure type):  59.11949685534591
Dataset Name:  disney , AUC Score(benchmark/combined):  0.6454802259887005
Dataset Name:  disney , AUC Score (contextual):  20.385674931129476
Dataset Name:  disney , AUC Score (structural):  36.391437308868504
Dataset Name:  disney , AUC Score (joint-type):  54.82093663911846
Datas

100%|██████████| 500/500 [00:16<00:00, 30.59it/s]


Dataset Name:  disney , AUC Score(benchmark/combined):  0.5466101694915254
Dataset Name:  disney , AUC Score (contextual):  20.385674931129472
Dataset Name:  disney , AUC Score (structural):  37.125382262996936
Dataset Name:  disney , AUC Score (joint-type):  25.6198347107438
Dataset Name:  disney , AUC Score (structure type):  34.329140461215935
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7754237288135593
Dataset Name:  disney  Best AUC Score (contextual):  51.790633608815426
Dataset Name:  disney  Best AUC Score (structural):  60.24464831804281
Dataset Name:  disney  Best AUC Score (joint-type):  25.6198347107438
Dataset Name:  disney  Best AUC Score (structure type):  59.11949685534591
Dataset Name:  disney , AUC Score(benchmark/combined):  0.594632768361582
Dataset Name:  disney , AUC Score (contextual):  33.88429752066116
Dataset Name:  disney , AUC Score (structural):  35.596330275229356
Dataset Name:  disney , AUC Score (joint-type):  31.955922865013775
Dataset

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="disney")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=5000)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.5) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])


print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  disney lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 5000 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.5 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 0/500 [00:00<?, ?it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.423728813559322


  1%|          | 4/500 [00:00<00:15, 32.01it/s]

Dataset Name:  disney , AUC Score (contextual):  58.4022038567493
Dataset Name:  disney , AUC Score (structural):  53.88379204892966
Dataset Name:  disney , AUC Score (joint-type):  21.48760330578512
Dataset Name:  disney , AUC Score (structure type):  47.55360087960418
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.423728813559322
Dataset Name:  disney  Best AUC Score (contextual):  58.4022038567493
Dataset Name:  disney  Best AUC Score (structural):  53.88379204892966
Dataset Name:  disney  Best AUC Score (joint-type):  21.48760330578512
Dataset Name:  disney  Best AUC Score (structure type):  47.55360087960418
Dataset Name:  disney , AUC Score(benchmark/combined):  0.4194915254237288
Dataset Name:  disney , AUC Score (contextual):  59.50413223140495
Dataset Name:  disney , AUC Score (structural):  53.88379204892966
Dataset Name:  disney , AUC Score (joint-type):  21.48760330578512
Dataset Name:  disney , AUC Score (structure type):  47.4986256184717
Dataset Name:  dis

...

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6313559322033898
Dataset Name:  disney , AUC Score (contextual):  39.66942148760331
Dataset Name:  disney , AUC Score (structural):  38.226299694189606
Dataset Name:  disney , AUC Score (joint-type):  46.831955922865006
Dataset Name:  disney , AUC Score (structure type):  40.021990104452996
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.75
Dataset Name:  disney  Best AUC Score (contextual):  81.267217630854
Dataset Name:  disney  Best AUC Score (structural):  55.77981651376146
Dataset Name:  disney  Best AUC Score (joint-type):  46.831955922865006
Dataset Name:  disney  Best AUC Score (structure type):  57.88894997251237
Dataset Name:  disney , AUC Score(benchmark/combined):  0.6497175141242938
Dataset Name:  disney , AUC Score (contextual):  38.84297520661157
Dataset Name:  disney , AUC Score (structural):  43.24159021406728
Dataset Name:  disney , AUC Score (joint-type):  43.526170798898065
Dataset Name:  disney

 99%|█████████▉| 494/500 [00:17<00:00, 27.63it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.673728813559322
Dataset Name:  disney , AUC Score (contextual):  51.515151515151516
Dataset Name:  disney , AUC Score (structural):  39.63302752293578
Dataset Name:  disney , AUC Score (joint-type):  43.526170798898065
Dataset Name:  disney , AUC Score (structure type):  41.39637163276526
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.75
Dataset Name:  disney  Best AUC Score (contextual):  81.267217630854
Dataset Name:  disney  Best AUC Score (structural):  55.77981651376146
Dataset Name:  disney  Best AUC Score (joint-type):  43.526170798898065
Dataset Name:  disney  Best AUC Score (structure type):  57.88894997251237
Dataset Name:  disney , AUC Score(benchmark/combined):  0.6652542372881356
Dataset Name:  disney , AUC Score (contextual):  39.39393939393939
Dataset Name:  disney , AUC Score (structural):  46.54434250764526
Dataset Name:  disney , AUC Score (joint-type):  48.484848484848484
Dataset Name:  disney ,

100%|██████████| 500/500 [00:17<00:00, 28.34it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6242937853107344
Dataset Name:  disney , AUC Score (contextual):  61.707988980716244
Dataset Name:  disney , AUC Score (structural):  41.40672782874618
Dataset Name:  disney , AUC Score (joint-type):  46.831955922865006
Dataset Name:  disney , AUC Score (structure type):  42.935678944474994
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.75
Dataset Name:  disney  Best AUC Score (contextual):  81.267217630854
Dataset Name:  disney  Best AUC Score (structural):  55.77981651376146
Dataset Name:  disney  Best AUC Score (joint-type):  46.831955922865006
Dataset Name:  disney  Best AUC Score (structure type):  57.88894997251237
Dataset Name:  disney , AUC Score(benchmark/combined):  0.6186440677966102
Dataset Name:  disney , AUC Score (contextual):  48.484848484848484
Dataset Name:  disney , AUC Score (structural):  43.730886850152906
Dataset Name:  disney , AUC Score (joint-type):  43.80165289256198
Dataset Name:  disne

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="disney")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=5000)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.5) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])


print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  disney lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 5000 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.5 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  1%|          | 6/500 [00:00<00:17, 27.62it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.5395480225988701
Dataset Name:  disney , AUC Score (contextual):  85.9504132231405
Dataset Name:  disney , AUC Score (structural):  65.68807339449542
Dataset Name:  disney , AUC Score (joint-type):  64.73829201101928
Dataset Name:  disney , AUC Score (structure type):  63.99120395821881
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.5395480225988701
Dataset Name:  disney  Best AUC Score (contextual):  85.9504132231405
Dataset Name:  disney  Best AUC Score (structural):  65.68807339449542
Dataset Name:  disney  Best AUC Score (joint-type):  64.73829201101928
Dataset Name:  disney  Best AUC Score (structure type):  63.99120395821881
Dataset Name:  disney , AUC Score(benchmark/combined):  0.53954802259887
Dataset Name:  disney , AUC Score (contextual):  85.67493112947659
Dataset Name:  disney , AUC Score (structural):  65.81039755351682
Dataset Name:  disney , AUC Score (joint-type):  64.18732782369145
Dataset Name: 

  2%|▏         | 12/500 [00:00<00:17, 28.08it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.5437853107344632
Dataset Name:  disney , AUC Score (contextual):  85.67493112947659
Dataset Name:  disney , AUC Score (structural):  65.565749235474
Dataset Name:  disney , AUC Score (joint-type):  63.91184573002755
Dataset Name:  disney , AUC Score (structure type):  63.71632765255636
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.5437853107344632
Dataset Name:  disney  Best AUC Score (contextual):  85.9504132231405
Dataset Name:  disney  Best AUC Score (structural):  65.81039755351682
Dataset Name:  disney  Best AUC Score (joint-type):  63.91184573002755
Dataset Name:  disney  Best AUC Score (structure type):  63.99120395821881
Dataset Name:  disney , AUC Score(benchmark/combined):  0.5466101694915255
Dataset Name:  disney , AUC Score (contextual):  85.39944903581268
Dataset Name:  disney , AUC Score (structural):  65.81039755351682
Dataset Name:  disney , AUC Score (joint-type):  63.91184573002755
Dataset Name:

...

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6214689265536724
Dataset Name:  disney , AUC Score (contextual):  23.966942148760328
Dataset Name:  disney , AUC Score (structural):  35.22935779816514
Dataset Name:  disney , AUC Score (joint-type):  27.823691460055095
Dataset Name:  disney , AUC Score (structure type):  33.6998350742166
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7796610169491525
Dataset Name:  disney  Best AUC Score (contextual):  85.9504132231405
Dataset Name:  disney  Best AUC Score (structural):  66.23853211009174
Dataset Name:  disney  Best AUC Score (joint-type):  27.823691460055095
Dataset Name:  disney  Best AUC Score (structure type):  64.26608026388125
Dataset Name:  disney , AUC Score(benchmark/combined):  0.6200564971751412
Dataset Name:  disney , AUC Score (contextual):  25.89531680440771
Dataset Name:  disney , AUC Score (structural):  39.38837920489296
Dataset Name:  disney , AUC Score (joint-type):  43.52617079889807
Dataset N

100%|█████████▉| 498/500 [00:17<00:00, 28.40it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6016949152542372
Dataset Name:  disney , AUC Score (contextual):  44.62809917355372
Dataset Name:  disney , AUC Score (structural):  43.24159021406728
Dataset Name:  disney , AUC Score (joint-type):  37.74104683195593
Dataset Name:  disney , AUC Score (structure type):  40.84661902144036
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7796610169491525
Dataset Name:  disney  Best AUC Score (contextual):  85.9504132231405
Dataset Name:  disney  Best AUC Score (structural):  66.23853211009174
Dataset Name:  disney  Best AUC Score (joint-type):  37.74104683195593
Dataset Name:  disney  Best AUC Score (structure type):  64.26608026388125
Dataset Name:  disney , AUC Score(benchmark/combined):  0.5338983050847458
Dataset Name:  disney , AUC Score (contextual):  65.0137741046832
Dataset Name:  disney , AUC Score (structural):  45.5045871559633
Dataset Name:  disney , AUC Score (joint-type):  60.33057851239669
Dataset Name:

100%|██████████| 500/500 [00:17<00:00, 27.90it/s]


Dataset Name:  disney , AUC Score(benchmark/combined):  0.57909604519774
Dataset Name:  disney , AUC Score (contextual):  65.56473829201101
Dataset Name:  disney , AUC Score (structural):  53.63914373088685
Dataset Name:  disney , AUC Score (joint-type):  48.760330578512395
Dataset Name:  disney , AUC Score (structure type):  50.522264980758656
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7796610169491525
Dataset Name:  disney  Best AUC Score (contextual):  85.9504132231405
Dataset Name:  disney  Best AUC Score (structural):  66.23853211009174
Dataset Name:  disney  Best AUC Score (joint-type):  48.760330578512395
Dataset Name:  disney  Best AUC Score (structure type):  64.26608026388125
Dataset Name:  disney , AUC Score(benchmark/combined):  0.5607344632768362
Dataset Name:  disney , AUC Score (contextual):  59.77961432506886
Dataset Name:  disney , AUC Score (structural):  48.99082568807339
Dataset Name:  disney , AUC Score (joint-type):  45.730027548209364
Dataset N

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="disney")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=5000)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.5) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])


print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  disney lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 5000 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.5 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 0/500 [00:00<?, ?it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.4223163841807909
Dataset Name:  disney , AUC Score (contextual):  83.4710743801653
Dataset Name:  disney , AUC Score (structural):  60.61162079510704
Dataset Name:  disney , AUC Score (joint-type):  44.3526170798898
Dataset Name:  disney , AUC Score (structure type):  58.01886792452831
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.4223163841807909
Dataset Name:  disney  Best AUC Score (contextual):  83.4710743801653
Dataset Name:  disney  Best AUC Score (structural):  60.61162079510704
Dataset Name:  disney  Best AUC Score (joint-type):  44.3526170798898
Dataset Name:  disney  Best AUC Score (structure type):  58.01886792452831


  1%|          | 6/500 [00:00<00:17, 27.93it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.4223163841807909
Dataset Name:  disney , AUC Score (contextual):  83.4710743801653
Dataset Name:  disney , AUC Score (structural):  60.67278287461774
Dataset Name:  disney , AUC Score (joint-type):  44.0771349862259
Dataset Name:  disney , AUC Score (structure type):  58.018867924528294
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.4223163841807909
Dataset Name:  disney  Best AUC Score (contextual):  83.4710743801653
Dataset Name:  disney  Best AUC Score (structural):  60.67278287461774
Dataset Name:  disney  Best AUC Score (joint-type):  44.0771349862259
Dataset Name:  disney  Best AUC Score (structure type):  58.01886792452831
Dataset Name:  disney , AUC Score(benchmark/combined):  0.42090395480225984
Dataset Name:  disney , AUC Score (contextual):  83.4710743801653
Dataset Name:  disney , AUC Score (structural):  60.73394495412844
Dataset Name:  disney , AUC Score (joint-type):  44.0771349862259
Dataset Name: 

...

Dataset Name:  disney , AUC Score(benchmark/combined):  0.7175141242937852
Dataset Name:  disney , AUC Score (contextual):  24.242424242424242
Dataset Name:  disney , AUC Score (structural):  51.74311926605505
Dataset Name:  disney , AUC Score (joint-type):  49.862258953168045
Dataset Name:  disney , AUC Score (structure type):  51.46750524109015
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.8757062146892656
Dataset Name:  disney  Best AUC Score (contextual):  83.4710743801653
Dataset Name:  disney  Best AUC Score (structural):  62.93577981651376
Dataset Name:  disney  Best AUC Score (joint-type):  49.862258953168045
Dataset Name:  disney  Best AUC Score (structure type):  60.220125786163514
Dataset Name:  disney , AUC Score(benchmark/combined):  0.652542372881356
Dataset Name:  disney , AUC Score (contextual):  39.6694214876033
Dataset Name:  disney , AUC Score (structural):  56.02446483180428
Dataset Name:  disney , AUC Score (joint-type):  60.05509641873279
Dataset N

100%|█████████▉| 498/500 [00:17<00:00, 28.00it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.6511299435028248
Dataset Name:  disney , AUC Score (contextual):  38.56749311294766
Dataset Name:  disney , AUC Score (structural):  55.16819571865443
Dataset Name:  disney , AUC Score (joint-type):  61.70798898071625
Dataset Name:  disney , AUC Score (structure type):  56.65618448637317
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.8757062146892656
Dataset Name:  disney  Best AUC Score (contextual):  83.4710743801653
Dataset Name:  disney  Best AUC Score (structural):  62.93577981651376
Dataset Name:  disney  Best AUC Score (joint-type):  61.70798898071625
Dataset Name:  disney  Best AUC Score (structure type):  60.220125786163514
Dataset Name:  disney , AUC Score(benchmark/combined):  0.67090395480226
Dataset Name:  disney , AUC Score (contextual):  55.6473829201102
Dataset Name:  disney , AUC Score (structural):  55.96330275229357
Dataset Name:  disney , AUC Score (joint-type):  55.37190082644627
Dataset Name:

100%|██████████| 500/500 [00:17<00:00, 27.86it/s]


Dataset Name:  disney , AUC Score(benchmark/combined):  0.6694915254237288
Dataset Name:  disney , AUC Score (contextual):  44.0771349862259
Dataset Name:  disney , AUC Score (structural):  46.727828746177366
Dataset Name:  disney , AUC Score (joint-type):  57.30027548209367
Dataset Name:  disney , AUC Score (structure type):  48.58490566037736
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.8757062146892656
Dataset Name:  disney  Best AUC Score (contextual):  83.4710743801653
Dataset Name:  disney  Best AUC Score (structural):  62.93577981651376
Dataset Name:  disney  Best AUC Score (joint-type):  57.30027548209367
Dataset Name:  disney  Best AUC Score (structure type):  60.220125786163514


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
parser = argparse.ArgumentParser(description='parameters')
parser.add_argument('-f')
parser.add_argument('--dataset', type=str, default="disney")
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--epoch_num', type=int, default=500)
parser.add_argument('--lambda_loss1', type=float, default=1e-2) #neighbor reconstruction loss weight
parser.add_argument('--lambda_loss2', type=float, default=0.1) #feature loss weight
parser.add_argument('--lambda_loss3', type=float, default=0.8) #degree loss weight
parser.add_argument('--sample_size', type=int, default=10)
parser.add_argument('--dimension', type=int, default=16)
parser.add_argument('--encoder', type=str, default="SAGE")
parser.add_argument('--loss_step', type=int, default=5000)
parser.add_argument('--real_loss', type=bool, default=False) #use real loss or adaptive loss
parser.add_argument('--neigh_loss', type=str, default="KL")
parser.add_argument('--h_loss_weight', type=float, default=1.0)#adaptive loss weight for h_loss
parser.add_argument('--feature_loss_weight', type=float, default=2.5) #adaptive loss weight for feature loss
parser.add_argument('--degree_loss_weight', type=float, default=1.0)#adaptive loss weight for degree loss
parser.add_argument('--calculate_contextual', type=bool, default=True)
parser.add_argument('--contextual_n', type=int, default=3)
parser.add_argument('--contextual_k', type=int, default=5)
parser.add_argument('--calculate_structural', type=bool, default=True)
parser.add_argument('--structural_n', type=int, default=3)
parser.add_argument('--structural_m', type=int, default=5)
parser.add_argument('--use_combine_outlier', type=bool, default=False)
parser.add_argument('--plot_loss', type=bool, default=False)
parser.add_argument('--normalize_feat', type=bool, default=False)



args = parser.parse_args([])


print("GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction")
print("Dataset: ", args.dataset, "lr:", args.lr, "lambda_loss1 (neighbor):",args.lambda_loss1, "lambda_loss2 (feature):", args.lambda_loss2, "lambda_loss3 (degree):", args.lambda_loss3, "sample_size:", args.sample_size, "dimension:",args.dimension, "encoder:", args.encoder, "loss_step:", args.loss_step,"real_loss:", args.real_loss, "h_loss_weight:",args.h_loss_weight,"feature_loss_weight",args.feature_loss_weight,"degree_loss_weight:",args.degree_loss_weight,
"calculate_contextual",args.calculate_contextual,"calculate_structural",args.calculate_structural)


dataset_str = args.dataset
train_real_datasets(dataset_str=dataset_str, lr=args.lr, epoch_num=args.epoch_num, lambda_loss1=args.lambda_loss1, 
lambda_loss2=args.lambda_loss2, lambda_loss3=args.lambda_loss3, encoder=args.encoder, sample_size=args.sample_size, loss_step=args.loss_step, 
hidden_dim=args.dimension,real_loss=args.real_loss,calculate_contextual=args.calculate_contextual,calculate_structural=args.calculate_structural)

GAD-NR: Graph Anomaly Detection via Neighborhood Reconstruction
Dataset:  disney lr: 0.01 lambda_loss1 (neighbor): 0.01 lambda_loss2 (feature): 0.1 lambda_loss3 (degree): 0.8 sample_size: 10 dimension: 16 encoder: SAGE loss_step: 5000 real_loss: False h_loss_weight: 1.0 feature_loss_weight 2.5 degree_loss_weight: 1.0 calculate_contextual True calculate_structural True


  0%|          | 0/500 [00:00<?, ?it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.4576271186440678
Dataset Name:  disney , AUC Score (contextual):  54.82093663911846
Dataset Name:  disney , AUC Score (structural):  53.33333333333332
Dataset Name:  disney , AUC Score (joint-type):  80.71625344352617
Dataset Name:  disney , AUC Score (structure type):  58.700209643605874
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.4576271186440678
Dataset Name:  disney  Best AUC Score (contextual):  54.82093663911846
Dataset Name:  disney  Best AUC Score (structural):  53.33333333333332
Dataset Name:  disney  Best AUC Score (joint-type):  80.71625344352617
Dataset Name:  disney  Best AUC Score (structure type):  58.700209643605874


  1%|          | 3/500 [00:00<00:19, 25.82it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.45621468926553665
Dataset Name:  disney , AUC Score (contextual):  54.82093663911846
Dataset Name:  disney , AUC Score (structural):  53.14984709480122
Dataset Name:  disney , AUC Score (joint-type):  80.71625344352617
Dataset Name:  disney , AUC Score (structure type):  58.54297693920336
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.4576271186440678
Dataset Name:  disney  Best AUC Score (contextual):  54.82093663911846
Dataset Name:  disney  Best AUC Score (structural):  53.33333333333332
Dataset Name:  disney  Best AUC Score (joint-type):  80.71625344352617
Dataset Name:  disney  Best AUC Score (structure type):  58.700209643605874
Dataset Name:  disney , AUC Score(benchmark/combined):  0.45621468926553665
Dataset Name:  disney , AUC Score (contextual):  54.82093663911846
Dataset Name:  disney , AUC Score (structural):  52.96636085626911
Dataset Name:  disney , AUC Score (joint-type):  80.71625344352617
Dataset

...

Dataset Name:  disney , AUC Score (joint-type):  34.71074380165289
Dataset Name:  disney , AUC Score (structure type):  44.44444444444444
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7570621468926554
Dataset Name:  disney  Best AUC Score (contextual):  67.21763085399449
Dataset Name:  disney  Best AUC Score (structural):  62.50764525993884
Dataset Name:  disney  Best AUC Score (joint-type):  34.71074380165289
Dataset Name:  disney  Best AUC Score (structure type):  60.534591194968556
Dataset Name:  disney , AUC Score(benchmark/combined):  0.615819209039548
Dataset Name:  disney , AUC Score (contextual):  54.54545454545454
Dataset Name:  disney , AUC Score (structural):  48.44036697247706
Dataset Name:  disney , AUC Score (joint-type):  32.231404958677686
Dataset Name:  disney , AUC Score (structure type):  45.28301886792453
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7570621468926554
Dataset Name:  disney  Best AUC Score (contextual):  67.217630853994

 99%|█████████▊| 493/500 [00:14<00:00, 35.29it/s]

Dataset Name:  disney , AUC Score(benchmark/combined):  0.521186440677966
Dataset Name:  disney , AUC Score (contextual):  39.39393939393939
Dataset Name:  disney , AUC Score (structural):  51.0703363914373
Dataset Name:  disney , AUC Score (joint-type):  43.25068870523416
Dataset Name:  disney , AUC Score (structure type):  49.63312368972746
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7570621468926554
Dataset Name:  disney  Best AUC Score (contextual):  67.21763085399449
Dataset Name:  disney  Best AUC Score (structural):  62.50764525993884
Dataset Name:  disney  Best AUC Score (joint-type):  43.25068870523416
Dataset Name:  disney  Best AUC Score (structure type):  60.534591194968556
Dataset Name:  disney , AUC Score(benchmark/combined):  0.5494350282485876
Dataset Name:  disney , AUC Score (contextual):  51.515151515151516
Dataset Name:  disney , AUC Score (structural):  56.574923547400616
Dataset Name:  disney , AUC Score (joint-type):  49.58677685950412
Dataset N

100%|██████████| 500/500 [00:15<00:00, 33.22it/s]


Dataset Name:  disney , AUC Score(benchmark/combined):  0.5621468926553672
Dataset Name:  disney , AUC Score (contextual):  53.16804407713498
Dataset Name:  disney , AUC Score (structural):  57.18654434250765
Dataset Name:  disney , AUC Score (joint-type):  52.61707988980716
Dataset Name:  disney , AUC Score (structure type):  56.65618448637317
Dataset Name:  disney  Best AUC Score(benchmark/combined):  0.7570621468926554
Dataset Name:  disney  Best AUC Score (contextual):  67.21763085399449
Dataset Name:  disney  Best AUC Score (structural):  62.50764525993884
Dataset Name:  disney  Best AUC Score (joint-type):  52.61707988980716
Dataset Name:  disney  Best AUC Score (structure type):  60.534591194968556
Dataset Name:  disney , AUC Score(benchmark/combined):  0.5324858757062146
Dataset Name:  disney , AUC Score (contextual):  59.50413223140496
Dataset Name:  disney , AUC Score (structural):  50.70336391437309
Dataset Name:  disney , AUC Score (joint-type):  42.97520661157024
Dataset N